## Tensor的索引

### 索引操作与NumPy非常类似，主要包含下标索引、表达式索引、使用torch.where()与Tensor.clamp()的选择性索引。

In [1]:
import torch

In [2]:
a = torch.Tensor([[0, 1], [2, 3]])
a

tensor([[0., 1.],
        [2., 3.]])

In [3]:
#根据下标进行索引
a[1]

tensor([2., 3.])

In [4]:
#根据下标进行索引
a[0, 1]

tensor(1.)

In [5]:
#表达式索引
#选择a中大于0的元素，返回和a相同大小的Tensor，符合条件的置True，否则置False
a > 0

tensor([[False,  True],
        [ True,  True]])

In [6]:
#表达式索引
#选择符合条件的元素并返回，等价于torch.masked_select(a, a>0)
a[a>0]

tensor([1., 2., 3.])

In [7]:
#表达式索引
#选择符合条件的元素并返回，返回结果为一维Tensor
torch.masked_select(a, a>0)

tensor([1., 2., 3.])

In [8]:
#表达式索引
#选择非0元素的坐标，并返回
#不存在zero()和iszero()
torch.nonzero(a)

tensor([[0, 1],
        [1, 0],
        [1, 1]])

In [9]:
#torch.where(condition, x, y)，满足condition的位置输出x，否则输出y
torch.where(a>1, 2,  -2)

tensor([[-2, -2],
        [ 2,  2]])

In [10]:
#torch.clamp(min, max)，把Tensor的最小值限制为min，最大值限制为max
a.clamp(1.5, 2.5)

tensor([[1.5000, 1.5000],
        [2.0000, 2.5000]])

In [11]:
#当Tensor元素为整型，而clamp中的min、max为浮点型时，返回结果仍为整型Tensor
b = torch.tensor([[0, 1], [2, 3]])
b

tensor([[0, 1],
        [2, 3]])

In [12]:
b.clamp(1.5, 2.5)

tensor([[1, 1],
        [2, 2]])

## Tensor的变形

***注意：在进行Tensor操作时，有些操作如transpose()、permute()等可能会把Tensor在内存中变得不连续，而有些操作如view()等是需要Tensor内存连续的，这种情况下需要使用contiguous()操作先将内存变为连续的。在PyTorch v0.4版本中增加了reshape()操作，可以看做是Tensor.contiguous().view()。***

### 1.调整Tensor的形状，元素总数相同：view()、resize()、reshape()
view()、resize()和reshape()函数可以在不改变Tensor数据的前提下任意改变Tensor的形状，**必须保证调整前后的元素总数相同**，并且调整前后共享内存，三者的作用基本相同。

In [13]:
a = torch.arange(1, 5.)
a

tensor([1., 2., 3., 4.])

In [14]:
#使用view()函数进行维度变换
b = a.view(2, 2)
b

tensor([[1., 2.],
        [3., 4.]])

！！！使用resize()函数会报错**"non-inplace resize is deprecated"**，因此进行Tensor维度变换时，推荐使用reshape()

In [15]:
#使用resize()函数进行维度变换
c = a.resize(4, 1)
c

D:\Anaconda\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


tensor([[1.],
        [2.],
        [3.],
        [4.]])

In [16]:
#使用reshape()函数进行维度变换
d = a.reshape(4, 1)
d

tensor([[1.],
        [2.],
        [3.],
        [4.]])

In [17]:
#改变了b、c、d中的一个元素，a也跟着改变了，说明两者共享内存
b[0, 0] = 5
c[1, 0] = 7
d[2, 0] = 9
a

tensor([5., 7., 9., 4.])

如果想要直接改变Tensor的尺寸，可以使用**resize_()原地操作函数**。在resize_()函数中，如果超过了原Tensor的大小则重新分配内存，多出部分置0，如果小于原Tensor大小则剩余的部分仍然会隐藏保留。

In [18]:
#使用resize_()函数进行维度变换，操作后原Tensor也会跟着改变
e = a.resize_(2,3)
e

tensor([[5.0000e+00, 7.0000e+00, 9.0000e+00],
        [4.0000e+00, 6.6461e+22, 1.6148e-07]])

In [19]:
#使用resize_()函数进行维度变换，操作后原Tensor也会跟着改变
a

tensor([[5.0000e+00, 7.0000e+00, 9.0000e+00],
        [4.0000e+00, 6.6461e+22, 1.6148e-07]])

### 2.各维度之间的变换：transpose()和permute()函数

transpose()函数可以将指定的两个维度的元素进行转置，而permute()函数则可以按照给定的维度进行维度变换。

In [20]:
a = torch.randn(2, 3, 4)
a

tensor([[[-0.0327, -0.3447,  0.4214,  0.2413],
         [ 1.1238, -0.0321,  0.6531,  0.1410],
         [-0.6475,  1.7653, -1.1136, -2.7269]],

        [[-0.4449, -1.2753,  0.9509,  1.1101],
         [ 0.1763,  1.8675, -0.4399,  0.2039],
         [-0.9646, -0.7947,  1.4310,  0.6590]]])

In [21]:
#使用transpose()函数，将第0维和第1维的元素进行转置
#转置前为2×3×4，转置后为3×2×4
a.transpose(0, 1)

tensor([[[-0.0327, -0.3447,  0.4214,  0.2413],
         [-0.4449, -1.2753,  0.9509,  1.1101]],

        [[ 1.1238, -0.0321,  0.6531,  0.1410],
         [ 0.1763,  1.8675, -0.4399,  0.2039]],

        [[-0.6475,  1.7653, -1.1136, -2.7269],
         [-0.9646, -0.7947,  1.4310,  0.6590]]])

In [22]:
#使用permute()函数，按照第2、1、0的维度顺序进行元素排列
#转置前为2×3×4，转置后为4×3×2
a.permute(2, 1, 0)

tensor([[[-0.0327, -0.4449],
         [ 1.1238,  0.1763],
         [-0.6475, -0.9646]],

        [[-0.3447, -1.2753],
         [-0.0321,  1.8675],
         [ 1.7653, -0.7947]],

        [[ 0.4214,  0.9509],
         [ 0.6531, -0.4399],
         [-1.1136,  1.4310]],

        [[ 0.2413,  1.1101],
         [ 0.1410,  0.2039],
         [-2.7269,  0.6590]]])

### 3.处理size为1的维度：squeeze()和unsqueeze()函数

在实际的应用中，经常需要增加或减少Tensor的维度，尤其是维度为1的情况，这时候可以使用squeeze()与unsqueeze()函数，前者用于去除size为1的维度，而后者则是将指定的维度的size变为1。

In [23]:
a = torch.arange(1, 4)
a

tensor([1, 2, 3])

In [24]:
a.shape

torch.Size([3])

In [25]:
#unsqueeze()函数把指定维度变为1
#将第0维为变为1，总维度变为1×3
b = a.unsqueeze(0)
b

tensor([[1, 2, 3]])

In [26]:
b.shape

torch.Size([1, 3])

In [27]:
#squeeze()函数去掉维度为1的指定维度，如果不是1则不起任何作用
c = b.squeeze(0)
c

tensor([1, 2, 3])

In [28]:
c.shape

torch.Size([3])

In [29]:
#squeeze()函数去掉维度为1的指定维度，如果不是1则不起任何作用
d = b.squeeze(1)
d

tensor([[1, 2, 3]])

In [30]:
d.shape

torch.Size([1, 3])

### 4.复制元素来扩展维度：expand()和expand_as()函数

有时需要采用复制元素的形式来扩展Tensor的维度，这时expand就派上用场了。expand()函数将size为1的维度复制扩展为指定大小，也可以使用expand_as()函数指定为示例Tensor的维度。

In [31]:
a = torch.randn(2, 2, 1)
a

tensor([[[-0.8054],
         [ 0.9030]],

        [[-0.8270],
         [-2.0015]]])

In [32]:
#使用expand()函数扩展size为1的维度，扩展方式为复制相应维度的元素
a.expand(2, 2, 3)

tensor([[[-0.8054, -0.8054, -0.8054],
         [ 0.9030,  0.9030,  0.9030]],

        [[-0.8270, -0.8270, -0.8270],
         [-2.0015, -2.0015, -2.0015]]])

In [33]:
b = torch.randn(2, 2, 4)
b

tensor([[[-0.6537, -0.4217, -0.2860, -0.2449],
         [ 0.5360, -0.7000,  1.5311,  1.9038]],

        [[-2.6845,  0.1181,  0.4077,  1.1690],
         [-0.0313, -0.2575,  0.0802, -2.5208]]])

In [34]:
#使用expand_as()函数时，两者所有size不是1的维度，其维度数必须相同，否则无法使用
a.expand_as(b)

tensor([[[-0.8054, -0.8054, -0.8054, -0.8054],
         [ 0.9030,  0.9030,  0.9030,  0.9030]],

        [[-0.8270, -0.8270, -0.8270, -0.8270],
         [-2.0015, -2.0015, -2.0015, -2.0015]]])